In [1]:
import os
import sys
import tarfile
import time
import urllib.request
import numpy as np
import pandas as pd
import pyprind

In [2]:
source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'

if os.path.exists(target):
    os.remove(target)


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024. ** 2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write(f'\r{int(percent)}% | {progress_size / (1024. ** 2):.2f} MB '
                     f'| {speed:.2f} MB/s | {duration:.2f} sec elapsed')
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

if not os.path.isdir('aclImdb'):
    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

df = pd.read_csv('movie_data.csv', encoding='utf-8')

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})



/tmp/ipykernel_9822/2480140289.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]],


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


In [3]:
print(df.head(3))
print(df.shape)

                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0
(50000, 2)


### Bag of Words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = count.fit_transform(raw_documents=df["review"][:3])

count.vocabulary_

{'in': 166,
 '1974': 1,
 'the': 320,
 'teenager': 315,
 'martha': 200,
 'moxley': 217,
 'maggie': 197,
 'grace': 139,
 'moves': 214,
 'to': 333,
 'high': 153,
 'class': 59,
 'area': 22,
 'of': 231,
 'belle': 36,
 'haven': 148,
 'greenwich': 142,
 'connecticut': 66,
 'on': 234,
 'mischief': 205,
 'night': 226,
 'eve': 106,
 'halloween': 144,
 'she': 289,
 'was': 359,
 'murdered': 220,
 'backyard': 31,
 'her': 151,
 'house': 161,
 'and': 16,
 'murder': 219,
 'remained': 266,
 'unsolved': 347,
 'twenty': 341,
 'two': 342,
 'years': 381,
 'later': 188,
 'writer': 379,
 'mark': 199,
 'fuhrman': 126,
 'christopher': 57,
 'meloni': 203,
 'who': 371,
 'is': 174,
 'former': 122,
 'la': 185,
 'detective': 80,
 'that': 319,
 'has': 146,
 'fallen': 112,
 'disgrace': 88,
 'for': 121,
 'perjury': 245,
 'simpson': 292,
 'trial': 336,
 'moved': 213,
 'idaho': 164,
 'decides': 76,
 'investigate': 171,
 'case': 51,
 'with': 374,
 'his': 155,
 'partner': 241,
 'stephen': 304,
 'weeks': 365,
 'andrew': 17

In [5]:
docs.toarray()

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 0, 0]])

### TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)

tfidf.fit_transform(count.fit_transform(df["review"][:3])).toarray()

array([[0.  , 0.04, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.05, 0.05, 0.05],
       [0.04, 0.  , 0.04, ..., 0.  , 0.  , 0.  ]])

### Cleaning text

In [7]:
import re
def preprocess_text(t):
    text = re.sub('<[^>]*>', '', t)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)

    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text



df['review'] = df['review'].apply(preprocess_text)


### Tokenizing documents

In [8]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(t):
    return [porter.stem(word) for word in t.split()]

tokenizer_porter('runners like running and thus they run')


['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

### Stop words

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/acp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

### Training a logistic regression model for document classification

In [11]:
X_train, y_train = df.loc[:25000, 'review'].values, df.loc[:25000, 'sentiment'].values
X_test, y_test = df.loc[25000:, 'review'].values, df.loc[25000:, 'sentiment'].values

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [13]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None],
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                    {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer],
                     'vect__use_idf':[False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 10.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f174f0ec700>,
                                              <function tokenizer_porter at 0x7f174f0ec790...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x7f174f0ec700>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

In [14]:
gs_lr_tfidf.best_params_

{'clf__C': 10.0,
 'clf__penalty': 'l2',
 'vect__ngram_range': (1, 1),
 'vect__stop_words': None,
 'vect__tokenizer': <function __main__.tokenizer(text)>}

In [15]:
gs_lr_tfidf.best_score_

0.8970842631473704

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

### Out-of-core learning

In [28]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower() \
           + ' '.join(emoticons).replace('-', ''))
    tokenized = [w for w in text.split() if w not in stop]

    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [29]:
# test the function
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [30]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [21]:
"""
Unfortunately, we can’t use CountVectorizer for out-of-core learning since it requires holding the
complete vocabulary in memory. Also, TfidfVectorizer needs to keep all the feature vectors of the
training dataset in memory to calculate the inverse document frequencies. However, another useful
vectorizer for text processing implemented in scikit-learn is HashingVectorizer. HashingVectorizer
is data-independent and makes use of the hashing trick via the 32-bit MurmurHash3 function by Austin
Appleby (you can find more information about MurmurHash at https://en.wikipedia.org/wiki/MurmurHash)
"""

'\nUnfortunately, we can’t use CountVectorizer for out-of-core learning since it requires holding the\ncomplete vocabulary in memory. Also, TfidfVectorizer needs to keep all the feature vectors of the\ntraining dataset in memory to calculate the inverse document frequencies. However, another useful\nvectorizer for text processing implemented in scikit-learn is HashingVectorizer. HashingVectorizer\nis data-independent and makes use of the hashing trick via the 32-bit MurmurHash3 function by Austin\nAppleby (you can find more information about MurmurHash at https://en.wikipedia.org/wiki/MurmurHash)\n'

In [31]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log_loss', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [32]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

In [33]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Accuracy: 0.868


In [34]:
clf = clf.partial_fit(X_test, y_test)

In [35]:
doc = get_minibatch(doc_stream, size=1)

### Topic modeling with latent Dirichlet allocation

LDA is a generative probabilistic model that tries to find groups of words that appear frequently together across different
documents. These frequently appearing words represent our topics, assuming
that each document is a mixture of different words. The input to an LDA is the bag-of-words model.

Given a bag-of-words matrix as input, LDA decomposes it into two new matrices:
 • A document-to-topic matrix
 • A word-to-topic matrix

LDA decomposes the bag-of-words matrix in such a way that if we multiply those two matrices together, we will be able to reproduce
the input, the bag-of-words matrix, with the lowest possible error. In practice, we are interested in those topics that LDA found in the
bag-of-words matrix. The only downside may be that we must define the number of topics beforehand—the number of topics is a
hyperparameter of LDA that has to be specified manually.

### LDA with scikit-learn

In [49]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('movie_data.csv', encoding='utf-8')
# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [50]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [51]:
lda.components_.shape

(10, 5000)

In [52]:
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [53]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
